In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn import linear_model
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
folders = glob.glob("../data/00_gaze_data/sub*")
folders.sort()
subjs = []
for f in folders:
    subjs.append(f[-6:])
print(len(subjs))    
subjs[:3]

condition_index = ['count trash']*20 + ['free viewing']*20
condition_index[:3]

40


['count trash', 'count trash', 'count trash']

In [3]:
list_of_dataframes = []

for i in range(len(subjs)): #use the first 20 subjs 1-20 -- condition1
    read_file_name = '../data/03_fixation_billboards_recall_recognition/'  + subjs[i] +'_fixations_billboards_recall_recognition.csv'
    df = pd.read_csv(read_file_name, index_col = 0)
    df['subject'] = subjs[i]
    df["condition"] = condition_index[i]
    list_of_dataframes.append(df)
    
large_df = pd.concat(list_of_dataframes)

#add in the binary fix feature
large_df['fix_binarized'] = large_df['fix_bill']
large_df.loc[large_df['fix_binarized'] > 1, 'fix_binarized'] = 1

print( 1- (large_df["recall"].sum()/len(large_df)))

large_df['recall'] = large_df['recall'].replace(0,'norecall')
large_df['recall'] = large_df['recall'].replace(1,'recall')

target = large_df["recall"]

data = large_df.drop(['recall', 'recognition', 'subject'], axis=1)
data.head()

0.765


,position_index,billboard_image_at_position_text,fix_bill,sum_fa,condition,fix_binarized
0,1,texting_driving,2.0,0,count trash,1.0
1,2,burger,1.0,0,count trash,1.0
2,3,drunk_driving,1.0,0,count trash,1.0
3,4,coffee,1.0,0,count trash,1.0
4,5,hotel,1.0,0,count trash,1.0


In [4]:
from sklearn.compose import make_column_selector as selector
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
categorical_columns

['billboard_image_at_position_text', 'condition']

In [5]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
categorical_encoded_data = encoder.fit_transform(data[categorical_columns])
categorical_encoded_data[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0.])

In [6]:
data_cat = pd.DataFrame(categorical_encoded_data)
data_cat.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
data = data.drop(['billboard_image_at_position_text', 'condition','fix_binarized', 'sum_fa'], axis=1)
data.head()

,position_index,fix_bill
0,1,2.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0


In [8]:
data_joined = np.hstack((data.values, data_cat.values))
data2 = data_joined
#plt.imshow(data[:50,:], vmin = 0, vmax = 1)
data2[10]

array([11.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.])

In [9]:
target.head()

0    norecall
1    norecall
2    norecall
3    norecall
4    norecall
Name: recall, dtype: object

In [10]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

# set up a dummy classifier
clf_dummy = DummyClassifier(random_state=42, strategy='uniform') 
clf_dummy.fit(data2, target)
y_pred = clf_dummy.predict(data2)


cv_results = cross_validate(clf_dummy, data2, target, cv=5, scoring="roc_auc")
scores = cv_results["test_score"]

print("The mean cross-validation accuracy (roc_auc) is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")
print("")


cv_results2 = cross_validate(clf_dummy, data2, target, cv=5, scoring="f1_weighted")
scores2 = cv_results2["test_score"]

print("The mean cross-validation accuracy (f1_weighted) is: "
      f"{scores2.mean():.3f} ± {scores2.std():.3f}")
print("")


print(classification_report(target, y_pred, target_names=clf_dummy.classes_))

The mean cross-validation accuracy (roc_auc) is: 0.500 ± 0.000

The mean cross-validation accuracy (f1_weighted) is: 0.487 ± 0.027

              precision    recall  f1-score   support

    norecall       0.76      0.48      0.59       612
      recall       0.23      0.52      0.32       188

    accuracy                           0.49       800
   macro avg       0.50      0.50      0.46       800
weighted avg       0.64      0.49      0.53       800



In [11]:

# set up Dummy CLF with stratified sampling
clf_dummy2 = DummyClassifier(random_state=42, strategy='stratified') 
clf_dummy2.fit(data2, target)
y_pred2 = clf_dummy2.predict(data2)


cv_results3 = cross_validate(clf_dummy2, data2, target, cv=5, scoring="roc_auc")
scores3 = cv_results3["test_score"]

print("The mean cross-validation accuracy (roc_auc) is: "
      f"{scores3.mean():.3f} ± {scores3.std():.3f}")
print("")


cv_results4 = cross_validate(clf_dummy2, data2, target, cv=5, scoring="f1_weighted")
scores4 = cv_results4["test_score"]

print("The mean cross-validation accuracy (f1_weighted) is: "
      f"{scores4.mean():.3f} ± {scores4.std():.3f}")
print("")


print(classification_report(target, y_pred2, target_names=clf_dummy.classes_))

The mean cross-validation accuracy (roc_auc) is: 0.452 ± 0.012

The mean cross-validation accuracy (f1_weighted) is: 0.606 ± 0.007

              precision    recall  f1-score   support

    norecall       0.77      0.76      0.77       612
      recall       0.25      0.25      0.25       188

    accuracy                           0.64       800
   macro avg       0.51      0.51      0.51       800
weighted avg       0.65      0.64      0.64       800



In [12]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', 
          class_weight='balanced', # penalize
          probability=True)

clf.fit(data2, target)
y_pred = clf.predict(data2)

cv_results = cross_validate(clf, data2, target, cv=5, scoring="roc_auc")
scores = cv_results["test_score"]

#print(scores)
print("The mean cross-validation accuracy (roc_auc) is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")
print("")

cv_results2 = cross_validate(clf, data2, target, cv=5, scoring="f1_weighted")
scores2 = cv_results2["test_score"]
#print(scores2)

print("The mean cross-validation accuracy (f1_weighted) is: "
      f"{scores2.mean():.3f} ± {scores2.std():.3f}")
print("")


print(classification_report(target, y_pred, target_names=clf.classes_))

The mean cross-validation accuracy (roc_auc) is: 0.728 ± 0.044

The mean cross-validation accuracy (f1_weighted) is: 0.580 ± 0.098

              precision    recall  f1-score   support

    norecall       0.89      0.58      0.71       612
      recall       0.36      0.77      0.49       188

    accuracy                           0.63       800
   macro avg       0.63      0.68      0.60       800
weighted avg       0.77      0.63      0.66       800



In [13]:
#optimization: 
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100, 1000]
}
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(data2, target)
best_params = grid_search.best_params_
best_params

{'C': 10}

In [ ]:
#optimization2
grid_search2 = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search2.fit(data2, target)
best_params2 = grid_search2.best_params_
best_params2

In [ ]:
#using paramter from optimization above
clf_optimized = SVC(C=10, kernel='linear', 
          class_weight='balanced', # penalize
          probability=True)
cv_results = cross_validate(clf_optimized, data2, target, cv=5, scoring="roc_auc")
scores = cv_results["test_score"]
#print(scores)
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")
cv_results2 = cross_validate(clf_optimized, data2, target, cv=5, scoring="f1_weighted")
scores2 = cv_results2["test_score"]
#print(scores2)
print("The mean cross-validation accuracy is: "
      f"{scores2.mean():.3f} ± {scores2.std():.3f}")
clf_optimized.fit(data2, target)
y_pred = clf_optimized.predict(data2)
print(classification_report(target, y_pred, target_names=clf_optimized.classes_))

In [ ]:
#using parameter from optimization above
clf_optimized2 = SVC(C=0.1, kernel='linear', 
          class_weight='balanced', # penalize
          probability=True)
cv_results = cross_validate(clf_optimized2, data2, target, cv=5, scoring="roc_auc")
scores = cv_results["test_score"]
#print(scores)
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")
cv_results2 = cross_validate(clf_optimized2, data2, target, cv=5, scoring="f1_weighted")
scores2 = cv_results2["test_score"]
#print(scores2)
print("The mean cross-validation accuracy is: "
      f"{scores2.mean():.3f} ± {scores2.std():.3f}")
clf_optimized2.fit(data2, target)
y_pred = clf_optimized2.predict(data2)
print(classification_report(target, y_pred, target_names=clf_optimized2.classes_))

### only dummy vs svc

In [ ]:
### DUMMY
clf_dummy = DummyClassifier(random_state=42, 
                            strategy='uniform')  #stratified, uniform, prior
clf_dummy.fit(data2, target)
y_pred = clf_dummy.predict(data2)

cv_results_dummy = cross_validate(clf_dummy, data2, target, cv=5, scoring="f1_weighted")
scores_dummy     = cv_results_dummy["test_score"]

print("The mean cross-validation accuracy for DUMMY (f1_weighted) is: " 
      f"{scores_dummy.mean():.3f} ± {scores_dummy.std():.3f}")
print("")

#print(classification_report(target, y_pred, target_names=clf_dummy.classes_))



### REAL
#using parameter from optimization above
clf_optimized = SVC(C=0.1, kernel='linear', 
                      class_weight='balanced', # penalize
                      probability=True)
clf_optimized.fit(data2, target)
y_pred = clf_optimized.predict(data2)

cv_results_svc = cross_validate(clf_optimized, data2, target, cv=5, scoring="f1_weighted")
scores_svc = cv_results_svc["test_score"]

print("The mean cross-validation accuracy for REAL SVC (f1_weighted) is: "
      f"{scores_svc.mean():.3f} ± {scores_svc.std():.3f}")

#print(classification_report(target, y_pred, target_names=clf_optimized2.classes_))